# Twitter Feeds Sentimental Analysis Project

##### Use Twitter APIs to retrieve tweets for sentimental test, a tweet with positive or negative sentiment.

In [1]:
#Import Libraries.

import pandas as pd
import numpy as np
import re
from textblob import TextBlob

##### Using Twython package for twitter API calls.

In [2]:
from twython import Twython
CONSUMER_KEY = #can be retrieved from twitter developer account
CONSUMER_SECRET = #can be retrieved from twitter developer account

twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET)

In [3]:
twitter

<Twython: JqNwizSUu16P4sBtO93j94bD6>

##### There are many variables in the API calls, they are the data we are receiving.

In [4]:
dir(Twython)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_error_message',
 '_request',
 'add_list_member',
 'construct_api_url',
 'create_block',
 'create_favorite',
 'create_friendship',
 'create_list',
 'create_list_members',
 'create_mute',
 'create_place',
 'create_saved_search',
 'cursor',
 'delete_list',
 'delete_list_member',
 'delete_list_members',
 'destroy_block',
 'destroy_direct_message',
 'destroy_favorite',
 'destroy_friendship',
 'destroy_mute',
 'destroy_saved_search',
 'destroy_status',
 'encode',
 'get',
 'get_account_settings',
 'get_application_rate_limit_status',
 'get_authentication_tokens',
 'get_authorized_tokens',
 'get_available_trends',
 'get_clo

In [5]:
help(Twython.search)

Help on function search in module twython.endpoints:

search(self, **params)
    Returns a collection of relevant Tweets matching a specified query.
    
    Docs: https://dev.twitter.com/docs/api/1.1/get/search/tweets



##### Searching the tweets with given keyword, specified tweet counts, language and specific area premises.

In [6]:
results = twitter.search(q="black panther", count=100, language='Eng', geocode='33.6845673,-117.82650490000003,100mi')
type(results)

dict

In [7]:
results

{'search_metadata': {'completed_in': 0.089,
  'count': 100,
  'max_id': 973438774070755329,
  'max_id_str': '973438774070755329',
  'next_results': '?max_id=973431083785183231&q=black%20panther&geocode=33.6845673%2C-117.82650490000003%2C100mi&count=100&include_entities=1',
  'query': 'black+panther',
  'refresh_url': '?since_id=973438774070755329&q=black%20panther&geocode=33.6845673%2C-117.82650490000003%2C100mi&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'},
 'statuses': [{'contributors': None,
   'coordinates': None,
   'created_at': 'Tue Mar 13 06:01:04 +0000 2018',
   'entities': {'hashtags': [{'indices': [13, 26], 'text': 'BlackPanther'}],
    'media': [{'display_url': 'pic.twitter.com/3403X789DB',
      'expanded_url': 'https://twitter.com/Variety/status/972524987910410241/photo/1',
      'id': 972522226766393344,
      'id_str': '972522226766393344',
      'indices': [111, 134],
      'media_url': 'http://pbs.twimg.com/media/DX8X0AaVMAAU5_V.jpg',
      'media_url_h

##### There are very big volume of data in each tweet, discovered the name of a key 'statuses' contains the actual text content of the tweet, that's what we need for analysis.

In [8]:
all = results['statuses']

##### Using a for loop to select and display all the tweets content retrieved from the API call.

In [9]:
for tweet in all:
    print(tweet['text'])
    print ("-----")

RT @Variety: #BlackPanther reached $1 billion at the global box office in just 26 days https://t.co/HOGQuHTqbn https://t.co/3403X789DB
-----
RT @vincestaples: Don’t let Black Panther’s success distract you from the fact that Michael B. Jordan let G Baby get popped. Some sins are…
-----
RT @shadowandact: For the first time in recent memory, two films directed and led by black talent are first and second at the box office. #…
-----
RT @shadowandact: For the first time in recent memory, two films directed and led by black talent are first and second at the box office. #…
-----
RT @shadowandact: For the first time in recent memory, two films directed and led by black talent are first and second at the box office. #…
-----
RT @lovebscott: Marvel President Kevin Feige Teases the Idea of a Standalone Trilogy Starring Black Panther's Okoye https://t.co/bGZumFqIJN
-----
GREAT Sunday @dubmagazine show w/ Black Panther Lexus &amp; activation for @wcccarcare available exclusively @Walmart d… https

##### Put these tweets into data frame for analysis.

In [10]:
tweet_df = pd.DataFrame(all)
tweet_df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,possibly_sensitive,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Tue Mar 13 06:01:04 +0000 2018,"{'hashtags': [{'text': 'BlackPanther', 'indice...","{'media': [{'id': 972522226766393344, 'id_str'...",0,False,None,973438774070755329,973438774070755329,...,False,NaN,NaN,9078,False,{'created_at': 'Sat Mar 10 17:30:00 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @Variety: #BlackPanther reached $1 billion ...,False,"{'id': 63959923, 'id_str': '63959923', 'name':..."
1,None,None,Tue Mar 13 06:00:01 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438512707112960,973438512707112960,...,NaN,NaN,NaN,17026,False,{'created_at': 'Sun Mar 11 04:57:38 +0000 2018...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @vincestaples: Don’t let Black Panther’s su...,False,"{'id': 3266788405, 'id_str': '3266788405', 'na..."
2,None,None,Tue Mar 13 05:59:49 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438460353830913,973438460353830913,...,NaN,NaN,NaN,10842,False,{'created_at': 'Sun Mar 11 20:33:24 +0000 2018...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @shadowandact: For the first time in recent...,False,"{'id': 1593614474, 'id_str': '1593614474', 'na..."
3,None,None,Tue Mar 13 05:59:12 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438307693576192,973438307693576192,...,NaN,NaN,NaN,10842,False,{'created_at': 'Sun Mar 11 20:33:24 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @shadowandact: For the first time in recent...,False,"{'id': 559750012, 'id_str': '559750012', 'name..."
4,None,None,Tue Mar 13 05:58:36 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438154739986437,973438154739986437,...,NaN,NaN,NaN,10842,False,{'created_at': 'Sun Mar 11 20:33:24 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @shadowandact: For the first time in recent...,False,"{'id': 14965505, 'id_str': '14965505', 'name':..."


In [11]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 29 columns):
contributors                 0 non-null object
coordinates                  0 non-null object
created_at                   100 non-null object
entities                     100 non-null object
extended_entities            6 non-null object
favorite_count               100 non-null int64
favorited                    100 non-null bool
geo                          0 non-null object
id                           100 non-null int64
id_str                       100 non-null object
in_reply_to_screen_name      0 non-null object
in_reply_to_status_id        0 non-null object
in_reply_to_status_id_str    0 non-null object
in_reply_to_user_id          0 non-null object
in_reply_to_user_id_str      0 non-null object
is_quote_status              100 non-null bool
lang                         100 non-null object
metadata                     100 non-null object
place                        4 non-nul

In [12]:
#[d['user'] for d in results['statuses']]

tweet_user = pd.DataFrame([d['user']for d in results['statuses']])
tweet_user.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,...,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Sat Aug 08 12:46:07 +0000 2009,False,False,Its Heaven and Earth. FYI by following this pe...,{'url': {'urls': [{'url': 'https://t.co/v9UoaA...,6931,None,1438,None,...,FFA04B,True,False,ItsJERRYlicious,43807,Hong Kong,none,https://t.co/v9UoaAT6Nb,28800.0,False
1,False,Fri Jul 03 04:19:53 +0000 2015,False,False,Saved by grace through faith! ...,{'url': {'urls': [{'url': 'https://t.co/Kj7VMP...,28223,None,273,None,...,000000,False,False,LightskinKai,25538,None,none,https://t.co/Kj7VMPrHb4,NaN,False
2,False,Sun Jul 14 15:14:03 +0000 2013,False,False,I'm everything that is difficult to describe,{'url': {'urls': [{'url': 'https://t.co/M6huEA...,1383,None,211,None,...,D94F5E,True,False,CallItLuv,5509,None,none,https://t.co/M6huEArLKW,NaN,False
3,False,Sat Apr 21 17:49:47 +0000 2012,False,False,I hate banana,{'url': {'urls': [{'url': 'https://t.co/rkrkQD...,212,None,154,None,...,843AA9,True,False,Khairil_mahfud,1288,Hawaii,none,https://t.co/rkrkQDH6uC,-36000.0,False
4,False,Sat May 31 19:12:44 +0000 2008,True,False,,{'description': {'urls': []}},7201,None,149,None,...,333333,True,False,plamm,4307,Central Time (US & Canada),none,None,-18000.0,False


In [13]:
df = tweet_df.merge(tweet_user, left_index=True, right_index=True)

In [14]:
df.head()

,contributors,coordinates,created_at_x,entities_x,extended_entities,favorite_count,favorited,geo,id_x,id_str_x,...,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,None,None,Tue Mar 13 06:01:04 +0000 2018,"{'hashtags': [{'text': 'BlackPanther', 'indice...","{'media': [{'id': 972522226766393344, 'id_str'...",0,False,None,973438774070755329,973438774070755329,...,FFA04B,True,False,ItsJERRYlicious,43807,Hong Kong,none,https://t.co/v9UoaAT6Nb,28800.0,False
1,None,None,Tue Mar 13 06:00:01 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438512707112960,973438512707112960,...,000000,False,False,LightskinKai,25538,None,none,https://t.co/Kj7VMPrHb4,NaN,False
2,None,None,Tue Mar 13 05:59:49 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438460353830913,973438460353830913,...,D94F5E,True,False,CallItLuv,5509,None,none,https://t.co/M6huEArLKW,NaN,False
3,None,None,Tue Mar 13 05:59:12 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438307693576192,973438307693576192,...,843AA9,True,False,Khairil_mahfud,1288,Hawaii,none,https://t.co/rkrkQDH6uC,-36000.0,False
4,None,None,Tue Mar 13 05:58:36 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,973438154739986437,973438154739986437,...,333333,True,False,plamm,4307,Central Time (US & Canada),none,None,-18000.0,False


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 71 columns):
contributors                          0 non-null object
coordinates                           0 non-null object
created_at_x                          100 non-null object
entities_x                            100 non-null object
extended_entities                     6 non-null object
favorite_count                        100 non-null int64
favorited                             100 non-null bool
geo                                   0 non-null object
id_x                                  100 non-null int64
id_str_x                              100 non-null object
in_reply_to_screen_name               0 non-null object
in_reply_to_status_id                 0 non-null object
in_reply_to_status_id_str             0 non-null object
in_reply_to_user_id                   0 non-null object
in_reply_to_user_id_str               0 non-null object
is_quote_status                       100 non-nu

##### 'text' column is the actual tweet content that we need for data analysis.

In [16]:
df['text']

0     RT @Variety: #BlackPanther reached $1 billion ...
1     RT @vincestaples: Don’t let Black Panther’s su...
2     RT @shadowandact: For the first time in recent...
3     RT @shadowandact: For the first time in recent...
4     RT @shadowandact: For the first time in recent...
5     RT @lovebscott: Marvel President Kevin Feige T...
6     GREAT Sunday @dubmagazine show w/ Black Panthe...
7     RT @THR: #BlackPanther is now the the No. 2 su...
8     RT @shadowandact: For the first time in recent...
9     RT @shadowandact: For the first time in recent...
10    RT @lovebscott: Marvel President Kevin Feige T...
11    RT @ComplexPop: Black Panther squad in Infinit...
12    RT @shadowandact: For the first time in recent...
13    RT @shadowandact: For the first time in recent...
14    RT @shadowandact: For the first time in recent...
15                 I want to watch black panther again😢
16    RT @shadowandact: For the first time in recent...
17    RT @shadowandact: For the first time in re

##### Data cleaning, by triming all the signs, emojis etc. Because sentimental analysis tool can only analyze words.

In [17]:
clean_text = pd.DataFrame()
sentiment_test = pd.DataFrame()

for i in df['text']:
    tweets_text = i.encode('utf-8')
    tweets_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweets_text.decode('utf-8')).split())
    analysis = TextBlob(tweets_text)
    if analysis.sentiment.polarity > 0:
        sent = ['positive']
    elif analysis.sentiment.polarity == 0:
        sent = ['neutral']
    else:
        sent = ['negative']
    tw = [tweets_text]
    sentiment_test = sentiment_test.append(sent)
    clean_text = clean_text.append(tw)
    print (tweets_text, sent)

RT BlackPanther reached 1 billion at the global box office in just 26 days ['neutral']
RT Don t let Black Panther s success distract you from the fact that Michael B Jordan let G Baby get popped Some sins are ['positive']
RT For the first time in recent memory two films directed and led by black talent are first and second at the box office ['positive']
RT For the first time in recent memory two films directed and led by black talent are first and second at the box office ['positive']
RT For the first time in recent memory two films directed and led by black talent are first and second at the box office ['positive']
RT Marvel President Kevin Feige Teases the Idea of a Standalone Trilogy Starring Black Panther s Okoye ['negative']
GREAT Sunday show w Black Panther Lexus amp activation for available exclusively d ['positive']
RT BlackPanther is now the the No 2 superhero movie of all time just behind Marvel s ['negative']
RT For the first time in recent memory two films directed and led 

In [18]:
print(clean_text)
print(sentiment_test)

                                                    0
0   RT BlackPanther reached 1 billion at the globa...
0   RT Don t let Black Panther s success distract ...
0   RT For the first time in recent memory two fil...
0   RT For the first time in recent memory two fil...
0   RT For the first time in recent memory two fil...
0   RT Marvel President Kevin Feige Teases the Ide...
0   GREAT Sunday show w Black Panther Lexus amp ac...
0   RT BlackPanther is now the the No 2 superhero ...
0   RT For the first time in recent memory two fil...
0   RT For the first time in recent memory two fil...
0   RT Marvel President Kevin Feige Teases the Ide...
0              RT Black Panther squad in Infinity War
0   RT For the first time in recent memory two fil...
0   RT For the first time in recent memory two fil...
0   RT For the first time in recent memory two fil...
0                 I want to watch black panther again
0   RT For the first time in recent memory two fil...
0   RT For the first time in

##### The index of the 2 DF is incorrect, need to be reindex.

In [19]:
clean_text = clean_text.reset_index()
sentiment_test = sentiment_test.reset_index()

print(clean_text)
print(sentiment_test)

    index                                                  0
0       0  RT BlackPanther reached 1 billion at the globa...
1       0  RT Don t let Black Panther s success distract ...
2       0  RT For the first time in recent memory two fil...
3       0  RT For the first time in recent memory two fil...
4       0  RT For the first time in recent memory two fil...
5       0  RT Marvel President Kevin Feige Teases the Ide...
6       0  GREAT Sunday show w Black Panther Lexus amp ac...
7       0  RT BlackPanther is now the the No 2 superhero ...
8       0  RT For the first time in recent memory two fil...
9       0  RT For the first time in recent memory two fil...
10      0  RT Marvel President Kevin Feige Teases the Ide...
11      0             RT Black Panther squad in Infinity War
12      0  RT For the first time in recent memory two fil...
13      0  RT For the first time in recent memory two fil...
14      0  RT For the first time in recent memory two fil...
15      0               

##### Creating dataframes and naming the columns, to collect final results.

In [20]:
clean_text = clean_text.rename(columns={0: "Tweet Content"})
del clean_text['index']
sentiment_test = sentiment_test.rename(columns={0: "Sentiment Test"})
del sentiment_test['index']


In [21]:
print(clean_text)
print(sentiment_test)

                                        Tweet Content
0   RT BlackPanther reached 1 billion at the globa...
1   RT Don t let Black Panther s success distract ...
2   RT For the first time in recent memory two fil...
3   RT For the first time in recent memory two fil...
4   RT For the first time in recent memory two fil...
5   RT Marvel President Kevin Feige Teases the Ide...
6   GREAT Sunday show w Black Panther Lexus amp ac...
7   RT BlackPanther is now the the No 2 superhero ...
8   RT For the first time in recent memory two fil...
9   RT For the first time in recent memory two fil...
10  RT Marvel President Kevin Feige Teases the Ide...
11             RT Black Panther squad in Infinity War
12  RT For the first time in recent memory two fil...
13  RT For the first time in recent memory two fil...
14  RT For the first time in recent memory two fil...
15                I want to watch black panther again
16  RT For the first time in recent memory two fil...
17  RT For the first time in

##### Reindexing the data frames.

In [22]:
clean_text = clean_text.reset_index()
sentiment_test = sentiment_test.reset_index()

In [23]:
clean_text = clean_text.rename(columns={0: "Tweet"},inplace = False)
del clean_text['index']
sentiment_test = sentiment_test.rename(columns={0:'Sentiment Test'}, inplace =False)
del sentiment_test['index']

##### Concatenate 2 data frames for the final result.

In [24]:
tweet_sentiment = pd.concat([clean_text, sentiment_test], axis=1)

In [25]:
tweet_sentiment.head()

,Tweet Content,Sentiment Test
0,RT BlackPanther reached 1 billion at the globa...,neutral
1,RT Don t let Black Panther s success distract ...,positive
2,RT For the first time in recent memory two fil...,positive
3,RT For the first time in recent memory two fil...,positive
4,RT For the first time in recent memory two fil...,positive


##### Drop all the duplicates data.

In [26]:
tweet_sentiment = tweet_sentiment.drop_duplicates()
tweet_sentiment.head()

,Tweet Content,Sentiment Test
0,RT BlackPanther reached 1 billion at the globa...,neutral
1,RT Don t let Black Panther s success distract ...,positive
2,RT For the first time in recent memory two fil...,positive
5,RT Marvel President Kevin Feige Teases the Ide...,negative
6,GREAT Sunday show w Black Panther Lexus amp ac...,positive
